In [1]:
from pandas import read_csv as rc
from pandas import DataFrame as df
from tqdm import tqdm
import numpy as np
import pandas as pd
from statsmodels.tsa.stattools import adfuller
from datetime import datetime
import matplotlib.pyplot as plt
from statsmodels.graphics.tsaplots import plot_acf
from statsmodels.graphics.tsaplots import plot_pacf
from pmdarima.arima import auto_arima
from feature_calculations import *
import statsmodels.api as sm
from statsmodels.tsa.arima.model import ARIMA
from scipy.stats import pearsonr

In [2]:
df_dict = {}

for asset_id in range(1, 2):
    df_dict[f'Asset_{asset_id}'] = rc(f"/Users/ryanlucas/Desktop/G-Research/Data/Asset-{asset_id}.csv")


In [3]:
features_dict = {}

for asset_id in range(1,2):
    features_dict[f'Asset_{asset_id}'] = rc(f"/Users/ryanlucas/Desktop/G-Research/Data/Asset-{asset_id}_features.csv")

In [4]:
features_dict['Asset_1'] = features_dict['Asset_1'].iloc[:, 4:]

In [5]:
df_dict[f'Asset_1']

,Unnamed: 0,timestamp,Count,Open,High,Low,Close,Volume,VWAP,Target
0,2,1514764860,229.0,13835.194000,14013.800000,13666.11,13850.176000,31.550062,13827.062093,-0.014643
1,10,1514764920,235.0,13835.036000,14052.300000,13680.00,13828.102000,31.046432,13840.362591,-0.015037
2,18,1514764980,528.0,13823.900000,14000.400000,13601.00,13801.314000,55.061820,13806.068014,-0.010309
3,26,1514765040,435.0,13802.512000,13999.000000,13576.28,13768.040000,38.780529,13783.598101,-0.008999
4,34,1514765100,742.0,13766.000000,13955.900000,13554.44,13724.914000,108.501637,13735.586842,-0.008079
...,...,...,...,...,...,...,...,...,...,...
1956277,24236739,1632182160,1940.0,42983.780000,43001.850849,42878.26,42899.012857,56.850913,42935.489499,NaN
1956278,24236753,1632182220,2026.0,42904.197143,42932.000000,42840.16,42860.005714,80.993326,42879.576084,NaN
1956279,24236767,1632182280,1986.0,42859.385714,42887.500000,42797.20,42827.020000,65.677734,42844.090693,NaN
1956280,24236781,1632182340,4047.0,42839.012802,43042.160000,42818.10,43017.277143,138.335477,42935.761938,NaN


In [6]:
def train_and_forecast_MA(data, ma_order= 1):

    model = ARIMA(data, order=(
        0, ma_order, 0)).fit(method="yule_walker")

    forecasts = model.forecast(1)

    return forecasts
    

In [9]:
def cross_validate(data, step_size, window_size, forecast_func = train_and_forecast_MA):
    forecasts = []
    actual_values = []
    for integer_position in tqdm(range(0, len(data)-window_size, step_size), position = 0, leave = True):
                        
        train_data = data.iloc[integer_position: integer_position+window_size]

        forecasts.append(float(forecast_func(train_data)))
        actual_value = data.iloc[integer_position+window_size+1]
        actual_values.append(float(actual_value))

    correlation, _ = pearsonr(forecasts, actual_values) 
    return correlation

In [11]:
window_size_dict = {}

for window_size in [2000, 3000, 5000, 10000, 20000, 40000, 100000, 500000]:
    correlation = cross_validate(df_dict['Asset_1']["Target"].fillna(method = 'bfill').fillna(method = 'ffill'),10000, window_size)
    window_size_dict[window_size] = correlation

 76%|███████▌  | 141/186 [02:54<01:02,  1.40s/it]

In [ ]:
df(window_size_dict, index = ["Correlation"]).transpose()

,Correlation
2000,0.624005
3000,0.692864
5000,0.750441
10000,0.775504
20000,0.774815
40000,0.777776
100000,0.783638
500000,0.815182
